In [14]:
import numpy as np
import pandas as pd
from scipy import stats

Question 3(a)

In [6]:
PO2 = pd.read_excel("/Users/utkarshtyagi/Documents/VSC/Jupyter/BT5126/Purchase Orders 2.xlsx")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
# Display the first few rows of the DataFrame
print(PO2.head())

            Supplier  Order No.  Item No.    Item Description  Item Cost  \
0    Hulkey Fasteners  Aug11001      1122  Airframe fasteners       4.25   
1       Alum Sheeting  Aug11002      1243  Airframe fasteners       4.25   
2  Fast-Tie Aerospace  Aug11003      5462  Shielded Cable/ft.       1.05   
3  Fast-Tie Aerospace  Aug11004      5462  Shielded Cable/ft.       1.05   
4       Steelpin Inc.  Aug11005      5319  Shielded Cable/ft.       1.10   

   Quantity  Cost per order  A/P Terms (Months) Order Date Arrival Date  \
0     19500         82875.0                  30 2011-08-05   2011-08-13   
1     10000         42500.0                  30 2011-08-08   2011-08-14   
2     23000         24150.0                  30 2011-08-10   2011-08-15   
3     21500         22575.0                  30 2011-08-15   2011-08-22   
4     17500         19250.0                  30 2011-08-20   2011-08-31   

   Arrival Time  
0             8  
1             6  
2             5  
3             7  
4 

In [9]:
# Filter orders with A/P Terms = 30 months
filtered = PO2[PO2['A/P Terms (Months)'] == 30]['Cost per order']

In [10]:
# Sample size, mean, standard deviation
n = filtered.count()
mean_cost = filtered.mean()
std_cost = filtered.std(ddof=1)  # sample standard deviation


In [11]:
# 90% confidence level
confidence = 0.90
alpha = 1 - confidence
t_critical = stats.t.ppf(1 - alpha/2, df=n - 1)


In [12]:
# Margin of error
margin_of_error = t_critical * (std_cost / np.sqrt(n))

# Confidence interval
ci_lower = mean_cost - margin_of_error
ci_upper = mean_cost + margin_of_error

In [13]:
print(f"Sample Mean Cost per Order: {mean_cost:.2f}")
print(f"90% Confidence Interval: ({ci_lower:.2f}, {ci_upper:.2f})")


Sample Mean Cost per Order: 28603.71
90% Confidence Interval: (22366.69, 34840.73)


compute the 90% confidence interval using t-distribution


Question 3 2nd part


In [15]:
# Total number of orders
n = len(PO2)

# Number of orders with Arrival Time < 6
count = (PO2['Arrival Time'] < 6).sum()

# Sample proportion
p_hat = count / n

In [16]:
# 95% confidence level
confidence = 0.95
alpha = 1 - confidence
z_critical = stats.norm.ppf(1 - alpha/2)


In [17]:
# Margin of error
margin_of_error = z_critical * np.sqrt(p_hat * (1 - p_hat) / n)

# Confidence interval
ci_lower = p_hat - margin_of_error
ci_upper = p_hat + margin_of_error

In [18]:
print(f"Sample Proportion: {p_hat:.3f}")
print(f"95% Confidence Interval: ({ci_lower:.3f}, {ci_upper:.3f})")

Sample Proportion: 0.266
95% Confidence Interval: (0.177, 0.355)


CI for proportion using Normal distribution as n=95 (large)

Interval Estimate-> Mean CI (90%)-> Parameter-> Mean cost per order for 30-month A/P terms, Sample used to estimate population mean
Interval Estimate-> Proportion CI(95%)->Parameter->Proportion of orders arriving <6 days , Sample used to estimate population proportion

Question 3(b)

In [19]:
# Total number of orders
n = len(PO2)

# Number of orders with Arrival Time < 6
count = (PO2['Arrival Time'] < 6).sum()

In [20]:
# Null proportion
p0 = 0.15

# Z-test statistic
z_stat = (p_hat - p0) / np.sqrt(p0 * (1 - p0) / n)

# One-tailed p-value (greater than)
p_value = 1 - stats.norm.cdf(z_stat)


In [21]:
print(f"Sample proportion: {p_hat:.3f}")
print(f"Z statistic: {z_stat:.3f}")
print(f"P-value (one-tailed): {p_value:.3f}")

Sample proportion: 0.266
Z statistic: 3.149
P-value (one-tailed): 0.001


Question 3(b) part 2


In [24]:
# Create a new column 'ap_gp'
PO2['ap_gp'] = PO2['A/P Terms (Months)'].apply(lambda x: 1 if x == 30 else 2)

# Check the first few rows
print(PO2[['A/P Terms (Months)', 'ap_gp']].head())

   A/P Terms (Months)  ap_gp
0                  30      1
1                  30      1
2                  30      1
3                  30      1
4                  30      1


In [25]:
# Group 1: ap_gp = 1 (30 months)
arrival_1 = PO2[PO2['ap_gp'] == 1]['Arrival Time']

# Group 2: ap_gp = 2 (15, 25, 45 months)
arrival_2 = PO2[PO2['ap_gp'] == 2]['Arrival Time']


In [26]:
# Two-sample t-test (Welch's t-test)
t_stat, p_value = stats.ttest_ind(arrival_1, arrival_2, equal_var=False)

print(f"T statistic: {t_stat:.3f}")
print(f"P-value: {p_value:.3f}")

T statistic: -1.257
P-value: 0.218


Question 3(b) part 3

In [27]:
def supplier_group(supplier):
    if supplier in ['Fast-Tie Aerospace', 'Hulkey Fasteners']:
        return 1
    elif supplier in ['Alum Sheeting', 'Durrable Products', 'Manley Valve']:
        return 2
    elif supplier in ['Steelpin Inc.', 'Spacetime Technologies', 'Pylon Accessories']:
        return 3
    else:
        return np.nan  # in case there are other suppliers


In [36]:
# Create the new column
PO2['s_gp'] = PO2['Supplier '].apply(supplier_group)



In [38]:
## Check the first few rows
print(PO2[['Supplier ', 's_gp', 'Arrival Time']].head())



            Supplier   s_gp  Arrival Time
0    Hulkey Fasteners     1             8
1       Alum Sheeting     2             6
2  Fast-Tie Aerospace     1             5
3  Fast-Tie Aerospace     1             7
4       Steelpin Inc.     3            11


In [39]:
# Group 1
arrival_1 = PO2[PO2['s_gp'] == 1]['Arrival Time']

# Group 2
arrival_2 = PO2[PO2['s_gp'] == 2]['Arrival Time']

# Group 3
arrival_3 = PO2[PO2['s_gp'] == 3]['Arrival Time']


In [41]:
from scipy.stats import f_oneway
# Perform ANOVA
f_stat, p_value = f_oneway(arrival_1, arrival_2, arrival_3)

print(f"F statistic: {f_stat:.3f}")
print(f"P-value: {p_value:.3f}")

F statistic: 18.960
P-value: 0.000


In [42]:
mean_arrival = PO2.groupby('s_gp')['Arrival Time'].mean()
print(mean_arrival)


s_gp
1     7.466667
2     5.968750
3    11.750000
Name: Arrival Time, dtype: float64
